In [ ]:
import pandas as pd
from astropy.table import Table
from astroquery.gaia import Gaia

In [ ]:
asPIC_ids = Table.read("../data/external/asPIC_1.1.fits")["sourceId"][
    :10000, 0
].tolist()

# Converting list to a format suitable for the SQL query
id_list = ",".join(["'{}'".format(id) for id in asPIC_ids])

In [ ]:
# get full gaia_source table and metallicity and alpha estimates from astrophysical_parameters

query = f"""
SELECT 
    gaiadr3.gaia_source.source_id,
    gaiadr3.gaia_source.parallax,
    gaiadr3.gaia_source.parallax_error,
    gaiadr3.gaia_source.ra,
    gaiadr3.gaia_source.ra_error,
    gaiadr3.gaia_source.dec,
    gaiadr3.gaia_source.dec_error,
    gaiadr3.gaia_source.pmra,
    gaiadr3.gaia_source.pmra_error,
    gaiadr3.gaia_source.pmdec,
    gaiadr3.gaia_source.pmdec_error,
    gaiadr3.gaia_source.radial_velocity,
    gaiadr3.gaia_source.radial_velocity_error,    
    astrophysical_parameters.mh_gspphot, 
    astrophysical_parameters.mh_gspphot_lower,
    astrophysical_parameters.mh_gspphot_upper, 
    astrophysical_parameters.mh_gspspec, 
    astrophysical_parameters.mh_gspspec_lower,
    astrophysical_parameters.mh_gspspec_upper, 
    astrophysical_parameters.alphafe_gspspec, 
    astrophysical_parameters.alphafe_gspspec_lower, 
    astrophysical_parameters.alphafe_gspspec_upper
FROM 
    gaiadr3.gaia_source
JOIN 
    gaiadr3.astrophysical_parameters
ON 
    gaiadr3.gaia_source.source_id = gaiadr3.astrophysical_parameters.source_id
WHERE 
    gaiadr3.gaia_source.source_id IN ({id_list});
"""

In [ ]:
# launch the job
job = Gaia.launch_job_async(query)
results = job.get_results()

In [ ]:
results.write("../data/raw/plato_targets.fits", format="fits")

In [ ]:
import polars as pl


total_rows = 174922161
# read xgboost.csv in batches and remove values with source_id not in asPIC_ids
dfs = []
for i in range(0, total_rows, int(1e7)):
    print(i)
    df = pl.read_csv(
        f"../data/external/xgboost.csv",
        has_header=True,
        columns=["source_id", "mh_xgboost"],
        skip_rows_after_header=i,
        n_rows=int(1e6),
    )
    df = df.filter(df["source_id"].is_in(asPIC_ids))
    dfs.append(df)

In [12]:
pl.read_csv("../data/external/xgboost.csv", n_rows=1).columns

['source_id',
 'catwise_w1',
 'catwise_w2',
 'in_training_sample',
 'mh_xgboost',
 'teff_xgboost',
 'logg_xgboost']

In [ ]:
import csv

with open("../data/external/xgboost.csv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for i, line in enumerate(reader):
        print("line[{}] = {}".format(i, line))

In [ ]:
import polars as pl

In [14]:
import polars as pl

# Define the path to your CSV file
file_path = "../data/external/xgboost.csv"

# Initialize the batched reader
reader = pl.read_csv_batched(
    source=file_path,
    # columns=["source_id", "mh_xgboost"],  # Only read the necessary columns
    batch_size=50000,  # Adjust based on your system's capabilities
)

# Prepare to store the filtered data
filtered_df = pl.DataFrame()

# Read batches until there are no more
batches = reader.next_batches()
while batches:
    # Concatenate the current batches into a single DataFrame
    df_current = pl.concat(batches)
    # Filter the DataFrame
    df_filtered = df_current.filter(pl.col("source_id").is_in(asPIC_ids))
    # Accumulate the filtered DataFrames
    filtered_df = pl.concat([filtered_df, df_filtered])
    # Read the next batch
    batches = reader.next_batches()

# At this point, filtered_df contains only the rows with source_ids in your reference list

TypeError: BatchedCsvReader.next_batches() missing 1 required positional argument: 'n'

In [18]:
import pandas as pd

# Initialize an empty DataFrame to store your filtered results
filtered_df = pd.DataFrame()

# Specify the chunk size
chunk_size = 50000  # You can adjust this size based on your system's memory capacity

# Use a loop to process the CSV in chunks
for chunk in pd.read_csv(
    file_path,
    chunksize=chunk_size,
):
    # Filter the chunk
    filtered_chunk = chunk[chunk["source_id"].isin(asPIC_ids)]
    # Append the filtered chunk to the DataFrame
    filtered_df = pd.concat([filtered_df, filtered_chunk])

# Now filtered_df contains only the rows with source_ids in your reference list

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

In [17]:
reader = pl.read_csv_batched(
    source=file_path,
    has_header=True,
    columns=["source_id", "mh_xgboost"],  # Only read the necessary columns
    batch_size=50000,  # Adjust based on your system's capabilities
)

thread 'python' panicked at crates/polars-io/src/csv/read_impl/batched_mmap.rs:315:78:
called `Result::unwrap()` on an `Err` value: ColumnNotFound(ErrString("unable to find column \"source_id\"; valid columns: [\"\\u{1f}�\\u{8}\\u{8}\\u{1f}��c\\0\\u{3}table-1.csv\\0��ٮ\", \"˲����gYg���\\u{5}�\\u{4}�'�\\u{12}�\\\"P*\"]"))


PanicException: called `Result::unwrap()` on an `Err` value: ColumnNotFound(ErrString("unable to find column \"source_id\"; valid columns: [\"\\u{1f}�\\u{8}\\u{8}\\u{1f}��c\\0\\u{3}table-1.csv\\0��ٮ\", \"˲����gYg���\\u{5}�\\u{4}�'�\\u{12}�\\\"P*\"]"))